In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
! pip install autopep8

In [ ]:
! pip install opencv-python

     |███████████████                 | 22.9 MB 2.7 MB/s eta 0:00:10

## Prerequisites

For this tutorial I reccomend you create new conda anvironment
```bash
conda create -n fast2 python=3.8.2 jupyter
```

## Install FastAI V2

In [ ]:
! pip install -U fastai2

## Import fastAI v2 vision modules

In [ ]:
from fastai2.vision.all import *

## Semantic segmentation with UNet model

Recall the UNet model:
<img src="images/od/unet.png" height="800" width="800">

Download the dataset.
#### Note Here I'll download the "tiny" version, but you should switch it with 
```python
untar_data(URLs.CAMVID)
```

In [ ]:
path = untar_data(URLs.CAMVID_TINY)

Let's look into the data

In [ ]:
path.ls()

In [ ]:
(path / 'images').ls()

In [ ]:
(path / 'labels').ls()

Load the class codes:

In [ ]:
codes = np.loadtxt(path/'codes.txt', dtype=str)
codes

In [ ]:
fnames = get_image_files(path/'images')
fnames[0]

In [ ]:
?? get_image_files

Initialize labels

In [ ]:
def label_func(fn): 
    return path/'labels'/f'{fn.stem}_P{fn.suffix}'

Initialize the data loader

In [ ]:
dls = SegmentationDataLoaders.from_label_func(path, bs=8, fnames=fnames, label_func=label_func, codes=codes, 
                                              seed=2020)

Look into the actual images

In [ ]:
dls.show_batch(max_n=8)

## Initialize the model

In [ ]:
learn = unet_learner(dls, resnet34)

Look in the model

In [ ]:
learn.model

Recall pixel-suffle up-sumpling layer:
<img src="images/od/pixel_shuffle_1.jpg" height="1000" width="1000">

Where are the concatenations?
<br/>
For this FastAi library uses so called runtime hooks

In [ ]:
# A simple hook class that returns the input and output of a layer during forward/backward pass
class Hook():
    def __init__(self, module, backward=False):
        if backward==False:
            self.hook = module.register_forward_hook(self.hook_fn)
        else:
            self.hook = module.register_backward_hook(self.hook_fn)
    def hook_fn(self, module, input, output):
        self.input = input
        self.output = output
    def close(self):
        self.hook.remove()

In [ ]:
print(learn.__dict__)

Check the optimizer

In [ ]:
learn.opt_func

Check the loss function:

In [ ]:
learn.loss_func

In [ ]:
?? FlattenedLoss

Check the type

In [ ]:
type(learn.loss_func)

In [ ]:
?? CrossEntropyLossFlat

In [ ]:
?? BaseLoss

Check the model performance:

In [ ]:
learn.show_results(max_n=6, figsize=(7, 8))

#### Fine-tune the model

In [ ]:
learn.fine_tune(8)

Check the results

In [ ]:
learn.show_results(max_n=6, figsize=(7, 8))

#### Save the model

In [ ]:
learn.save('unet_cv_ft', with_opt=True)

#### Copy and download the model weights

In [ ]:
(learn.path / learn.model_dir).ls()

In [ ]:
! cp  {learn.path / learn.model_dir / 'unet_cv_ft.pth'} /

In [ ]:
?? learn.save

#### Use data block API for image loading

In [ ]:
camvid = DataBlock(blocks=(ImageBlock, MaskBlock(codes)),
                   get_items = get_image_files,
                   get_y = label_func,
                   splitter=RandomSplitter(),
                   batch_tfms=aug_transforms(size=(120,160)))

In [ ]:
dls = camvid.dataloaders(path/"images", path=path, bs=8)

In [ ]:
dls.show_batch(max_n=8)

In [ ]:
?? get_image_files

In [ ]:
?? get_files

#### Get meta-data for model initialization

In [ ]:
dls.one_batch()[0].shape[-2:]

In [ ]:
get_c(dls)

## Find learning rate

In [ ]:
learn.lr_find()

fine-tuning means it will update all layers weights and biases with discriminative learning rate

In [ ]:
?? learn.fine_tune

#### Task: Try train with different size (progressive growing) and different augmentations

## Object detection with Detectron2 library

The library Detectron2 in Facebook's library on top of the PyTorch for fast object detection and semantic segmentation training

#### Install library

In [ ]:
! pip install -U cython 

In [ ]:
! pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

In [ ]:
! git clone https://github.com/facebookresearch/detectron2 detectron2_repo 
! pip install -e detectron2_repo

In [ ]:
! pip install -U watermark

In [2]:
%watermark

UsageError: Line magic function `%watermark` not found.


In [ ]:
%reload_ext watermark %watermark -v -p numpy, pandas, pycocotools, torch, torchvision, detectron2

#### Prepare dataset

In [ ]:
! wget https://storage.googleapis.com/kaggle-data-sets/34662/46346/bundle/archive.zip?Go

In [ ]:
! ls

In [ ]:
import pandas as pd

In [4]:
import torch, torchvision
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import glob

import os
import ntpath
import numpy as np
import cv2
import random
import itertools
import pandas as pd
from tqdm import tqdm
import urllib
import json
import PIL.Image as Image

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.structures import BoxMode

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

ModuleNotFoundError: No module named 'cv2'